<a href="https://colab.research.google.com/github/RuiQianSun/869/blob/main/Classification_Practice/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMA 869 Project: Decision Tree Practice

# Preliminaries: Inspect and Setup Environment

In [21]:
import pandas as pd
import numpy as np
import scipy
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

# Configure Notebook to show all outputs
InteractiveShell.ast_node_interactivity = "all"

# Preprocessing & Selection
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score

# Models: Linear & Neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Models: Tree-Based & Ensembles
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier

# Evaluation
from sklearn.metrics import accuracy_score, classification_report

In [22]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.6.1.


# 0: Read Data

In [23]:
df = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/resort_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   GuestID         6954 non-null   int64  
 1   BookingDate     6954 non-null   object 
 2   PromoCode       3709 non-null   object 
 3   Region          6785 non-null   object 
 4   AllInclusive    6786 non-null   float64
 5   Room            6568 non-null   object 
 6   PackageType     6801 non-null   object 
 7   Age             6478 non-null   float64
 8   VIP             6796 non-null   float64
 9   RoomService     6490 non-null   float64
 10  Dining          6466 non-null   float64
 11  Retail          6790 non-null   float64
 12  Spa             6806 non-null   float64
 13  Entertainment   6815 non-null   float64
 14  LoyaltyPoints   6954 non-null   int64  
 15  SurveyScore     6954 non-null   int64  
 16  DaysSinceEmail  6954 non-null   int64  
 17  BookingChannel  6954 non-null   o

* GuestID: identifies guest
* BookingDate: should convest to datatime object
* PromoCode should fill null values with "None"
* Region: some missing
* AllInclusive: why is this a float? some missing
* Churned: target, why int and not boolean?

In [24]:
df.head()

,GuestID,BookingDate,PromoCode,Region,AllInclusive,Room,PackageType,Age,VIP,RoomService,...,Retail,Spa,Entertainment,LoyaltyPoints,SurveyScore,DaysSinceEmail,BookingChannel,AgeGroup,ReferralSource,Churned
0,619623,2024-02-10,NaN,Americas,0.0,G/630/S,Relaxation,0.0,0.0,0.0,...,0.0,0.0,0.0,6915,5,136,Website,NaN,Facebook,1
1,776250,2024-01-03,NaN,Americas,1.0,G/201/S,Relaxation,17.0,0.0,0.0,...,0.0,0.0,0.0,8571,5,362,Corporate,Minor,Billboard,1
2,932709,2023-01-17,NaN,Americas,NaN,G/1483/S,Wellness,35.0,0.0,0.0,...,0.0,0.0,0.0,1142,4,154,TravelAgent,Middle,Facebook,0
3,771839,2023-12-09,PromoA,Europe,1.0,D/164/S,Adventure,26.0,0.0,0.0,...,0.0,NaN,0.0,9642,2,128,Website,Young,Magazine,1
4,755501,2024-02-15,PromoA,Americas,0.0,G/818/P,Relaxation,13.0,0.0,0.0,...,60.0,1.0,5147.0,5528,4,35,Mobile,Minor,Google,0


In [25]:
# Let's print some descriptive statistics for all the numeric features.

df.describe().T

,count,mean,std,min,25%,50%,75%,max
GuestID,6954.0,544815.121369,258449.781823,100000.0,322043.25,542402.5,769552.75,999943.000000
AllInclusive,6786.0,0.358974,0.479735,0.0,0.00,0.0,1.00,1.000000
Age,6478.0,28.682001,14.530100,0.0,19.00,27.0,38.00,79.000000
VIP,6796.0,0.023396,0.151169,0.0,0.00,0.0,0.00,1.000000
RoomService,6490.0,3823.687190,26416.086096,0.0,0.00,0.0,84.00,282324.947439
Dining,6466.0,7651.359101,54005.989936,0.0,0.00,0.0,95.00,590932.008676
Retail,6790.0,5980.799434,43130.400968,0.0,0.00,0.0,37.75,468882.332928
Spa,6806.0,5728.859525,41105.729649,0.0,0.00,0.0,80.00,447858.846683
Entertainment,6815.0,6757.871818,46162.309423,0.0,0.00,0.0,67.00,482032.266355
LoyaltyPoints,6954.0,5037.915876,2870.166683,0.0,2566.25,5055.5,7496.75,9996.000000


In [26]:
# Let's print some descriptive statistics for all the categorical features.
# What is the number of unique values in all the categorical features? And what is the value with the highest frequency?

df.describe(include=object).T

,count,unique,top,freq
BookingDate,6954,658,2024-01-20,59
PromoCode,3709,2,PromoB,1884
Region,6785,3,Americas,3673
Room,6568,5297,E/13/S,7
PackageType,6801,3,Relaxation,4694
BookingChannel,6954,5,Phone,1438
AgeGroup,6671,5,Young,2586
ReferralSource,6954,19,Google,1235


In [27]:
# How much missing data is in each feature?

df.isna().sum()

,0
GuestID,0
BookingDate,0
PromoCode,3245
Region,169
AllInclusive,168
Room,386
PackageType,153
Age,476
VIP,158
RoomService,464


In [28]:
# For convienience, let's save the names of all numeric features to a list, and the names of all categorical features to another list.

numeric_features = df.select_dtypes(include='number').columns.tolist()
print(numeric_features)

categorical_features = df.select_dtypes(include='object').columns.tolist()
print(categorical_features)

['GuestID', 'AllInclusive', 'Age', 'VIP', 'RoomService', 'Dining', 'Retail', 'Spa', 'Entertainment', 'LoyaltyPoints', 'SurveyScore', 'DaysSinceEmail', 'Churned']
['BookingDate', 'PromoCode', 'Region', 'Room', 'PackageType', 'BookingChannel', 'AgeGroup', 'ReferralSource']


In [29]:
# Do more EDA

# 1: Pipeline 1: Simple Feature Engineering and then Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [31]:
# Scikit-learn needs us to put the features in one dataframe, and the label in another.
# It's tradition to name these variables X and y, but it doesn't really matter.

X = df.drop(['GuestID', 'Churned'], axis=1)
y = df['Churned']

## 1.1: Cleaning and FE for Decision Tree

In [32]:
# We know this dataset has categorical features, and we also know that DTs don't
# allow categorical features. For now, we'll just remove (i.e., drop) these
# features.
#
# TODO: do something better, like encode them (as discussed in the course)

# remove catgorical features
X = X.drop(categorical_features, axis=1, errors='ignore')

In [33]:
# We know this dataset has some missing data, and we also know that DTs don't
# allow missing data. For now, we'll just do simple imputation.
#
# TODO: consider doing something different/better, like impute them (as
# discussed in class)

# replaces missing values with the mean of that feature
imp = SimpleImputer()
imp.fit(X)
X = imp.transform(X)

SimpleImputer()

In [34]:
# Do more data cleaning and FE

## 1.2: Model creation, hyperparameter tuning, and validation

In [35]:
# Let's create a very simple DecisionTree.

clf = DecisionTreeClassifier(max_depth=3, random_state=0)

# TODO: Can try different algorithms

In [36]:
# We use cross_validate to perform K-fold cross validation for us.
cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_macro")

# TODO: can also add hyperparameter tuning to explore different values of the algorithms
# hyperparameters, and see how much those affect results.
# See GridSearchCV or RandomizedSearchCV.

In [37]:
# Now that cross validation has completed, we can see what it estimates the peformance
# of our model to be.

display(cv_results)
print("The mean CV score is:")
print(np.mean(cv_results['test_score']))

{'fit_time': array([0.01388168, 0.01268888, 0.01377368, 0.01386404, 0.01411605]),
 'score_time': array([0.00358987, 0.00325108, 0.00363994, 0.00577831, 0.00328207]),
 'test_score': array([0.72808189, 0.7274893 , 0.72269741, 0.72117924, 0.71202368])}

The mean CV score is:
0.7222943022058785



Once we are happy with the estimated performance of our model, we can move on to the final step.

First, we train our model one last time, using all available training data (unlike CV, which always uses a subset). This final training will give our model the best chance as the highest performance.

Then, we must load in the (unlabeled) competition data from the cloud and use our model to generate predictions for each instance in that data. We will then output those predictions to a CSV file and upload it to the competition.

In [38]:
# Our model's "final form"

clf = clf.fit(X, y)

In [39]:
X_comp = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/resort_test.csv")

# Will need to save these IDs for later
passengerIDs = X_comp["GuestID"]

# Importantly, we need to perform the same cleaning/transformation steps
# on this competition data as you did the training data. Otherwise, we will
# get an error and/or unexpected results.

X_comp = X_comp.drop(['GuestID'], axis=1, errors='ignore')
X_comp = X_comp.drop(categorical_features, axis=1, errors='ignore')

X_comp = imp.transform(X_comp)

# Use your model to make predictions
pred_comp = clf.predict(X_comp)

# Create a simple dataframe with two columns: the passenger ID (just the same as the test data) and our predictions
my_submission = pd.DataFrame({
    'GuestID': passengerIDs,
    'Churned': pred_comp})

# Let's take a peak at the results (as a sanity check)
display(my_submission.head(10))

# You could use any filename.
my_submission.to_csv('submission.csv', index=False)

# You can now download the 'submission.csv' from Colab/Kaggle (see menu on the left or right)

,GuestID,Churned
0,154038,1
1,620160,1
2,655103,0
3,126993,1
4,635228,0
5,844514,1
6,541503,1
7,787572,0
8,645651,0
9,849608,1
